In [71]:
import os
import cv2
import mediapipe as mp
import time
from collections import namedtuple
from gtts import gTTS
import pygame

In [72]:
Point = namedtuple('Point', 'x y')
language = 'en'

In [73]:
class Line:
    def __init__(self, P, Q):
        self.a = P.y - Q.y
        self.b = Q.x - P.x
        self.c = P.x*(Q.y - P.y) - P.y*(Q.x - P.x)

    def substitute(self, R):
        return self.a*R.x + self.b*R.y + self.c

In [74]:
morse_to_text = {
        "" : "",
        ".-" : "A",
        "-..." : "B",
        "-.-." : "C",
        "-.." : "D",
        "." : "E",
        "..-." : "F",
        "--." : "G",
        "...." : "H",
        ".." : "I",
        ".---" : "J",
        "-.-" : "K",
        ".-.." : "L",
        "--" : "M",
        "-." : "N",
        "---" : "O",
        ".--." : "P",
        "--.-" : "Q",
        ".-." : "R",
        "..." : "S",
        "-" : "T",
        "..-" : "U",
        "...-" : "V",
        ".--" : "W",
        "-..-" : "X",
        "-.--" : "Y",
        "--.." : "Z",
    }

In [75]:
opened_hand = False
open_time = 0
close_time = 0
check_if_open = True
check_if_close = False
is_open = True
status = "CLOSED"
idle_time_start = 0
idle_time_end = 0

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

_fps_previous_time = 0
_fps_current_time = 0

In [76]:
morse = ""

cap = cv2.VideoCapture(cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    h, w, c = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:

            A = Point(int(handLms.landmark[0].x*w), int(handLms.landmark[0].y*h))
            B = Point(int(handLms.landmark[5].x*w), int(handLms.landmark[5].y*h))
            C = Point(int(handLms.landmark[17].x*w), int(handLms.landmark[17].y*h))
            P = Point(int(handLms.landmark[12].x*w), int(handLms.landmark[12].y*h))

            l1 = Line(A, B)
            l2 = Line(B, C)
            l3 = Line(C, A)

            if l1.substitute(P) > 0 and l2.substitute(P) > 0 and l3.substitute(P) > 0:
                status = "CLOSED"
            else:
                status = "OPEN"
                
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
            cv2.putText(frame, status, (10, 300), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 3, 1)

    _fps_current_time = time.time()
    fps = 1/(_fps_current_time - _fps_previous_time)
    _fps_previous_time = _fps_current_time
    cv2.putText(frame, str(int(fps)), (10, 30), cv2.FONT_HERSHEY_PLAIN, 1.7, (0, 0, 255), 3, 1)

    if status == "OPEN" and check_if_open:
        opened_hand = True
    elif status == "CLOSED" and check_if_close:
        is_open = False
        opened_hand = False
        check_if_close = False

    if opened_hand:
        open_time = time.time()
        
        if idle_time_start:
            idle_time_end = time.time()
        
        if idle_time_end - idle_time_start > 2:
            morse += "    "
        elif idle_time_end - idle_time_start > 1:
            morse += " "

        opened_hand = False
        check_if_open = False
        check_if_close = True
        is_open = True
    elif not is_open:
        is_open = True
        idle_time_start = close_time = time.time()
        timeopen = close_time - open_time

        if timeopen < 0.7:
            morse += "."
        else :
            morse += "-"
        check_if_open = True


    cv2.imshow("Video", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break


cap.release()
cv2.destroyAllWindows()
print(morse)
word = ""
for i in morse.split("    "):
    letter = ""
    for j in i.split(" "):
        letter += morse_to_text[j]
    word += letter
print(word)

.... ..
HI


In [82]:
text = "Hello World"
pygame.init()
tts = gTTS(text= word, lang= language, slow= False)
tts.save("Hw.mp3")
pygame.mixer.Sound('Hw.mp3').play()